In [25]:
# This is a python script that takes a large PDF file and splits it into smaller chunks.
# Then it converts each chunk into markdown format, by sending the text to OpenAI's GPT model.

# Import libraries
import os
import openai
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import base64
import datetime
import cv2
import numpy as np
from google.cloud import vision

POPPLER_PATH = r"C:\Program Files\poppler-24.08.0\Library\bin"

In [24]:
def extract_text_with_google_vision(image_path):
    """
    Use Google Cloud Vision OCR to extract text from the image.
    """
    client = vision.ImageAnnotatorClient()
    with open(image_path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    output = ""
    vertices_list = []

    # print("Texts:")
    for text in texts:
        # print(f'\n"{text.description}"')
        vertices = text.bounding_poly.vertices
        # print("Vertices:")
        for vertex in vertices:
            # print(f"({vertex.x}, {vertex.y})")
            # Convert to a list of tuples
            vertices_list.append((vertex.x, vertex.y))
        vertices_list.append(vertices)
        
        output += text.description + "\n"

    # print(f"Full text: {output}")

    return output, vertices_list

In [3]:
def convert_ocr_text_and_image_to_markdown(ocr_text, image_file_path, client, log_dir=None):
    """
    Sends both the OCR text and the original image to ChatGPT, instructing it to only arrange the OCR text into markdown,
    not to transcribe from the image.
    """
    with open(image_file_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode("utf-8")

    column_headers = [
        "ID",
        "Name",
        "Crew",
        "Daily Output",
        "Labor-Hours",
        "Unit",
        "Material",
        "Labor",
        "Equipment",
        "Total",
        "Total Incl O&P",
    ]

    prompt = (
        "You are given the OCR-extracted text from an image and the original image itself. "
        "Your task is to arrange the provided OCR text into markdown format, preserving any tables or structure. "
        "Do NOT transcribe or extract any new information from the image. "
        "Only use the provided OCR text. "
        "If the OCR text is unclear, leave it as is. "
        "Do not add, guess, or hallucinate any information. "
        "If a table contains blank or empty cells, preserve them as blank in the markdown table (do not fill or merge them). "
        "Keep the table structure and number of columns/rows as in the original text, even if some cells are empty. "
        f"The table columns are: {', '.join(column_headers)}. "
        "Provide only the markdown output, without any extra commentary or code blocks.\n\n"
        "OCR Text:\n"
        f"{ocr_text}"
    )

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        temperature=0,
    )
    markdown_text = response.choices[0].message.content.strip()

    # Logging
    if log_dir is not None:
        log_file_path = os.path.join(log_dir, "conversion_log.txt")
        with open(log_file_path, 'a', encoding='utf-8') as log_file:
            log_file.write(f"Timestamp: {datetime.datetime.now().isoformat()}\n")
            log_file.write(f"Image: {image_file_path}\n")
            log_file.write("OCR Text:\n")
            log_file.write(ocr_text + "\n")
            log_file.write("Prompt:\n")
            log_file.write(prompt + "\n")
            log_file.write("GPT Markdown Output:\n")
            log_file.write(markdown_text + "\n")
            log_file.write("="*60 + "\n\n")

    return markdown_text

In [4]:
def save_markdown(markdown_text, output_path):
    """
    Saves the markdown text to a file.
    
    Args:
        markdown_text (str): The markdown text to be saved.
        output_path (str): The path where the markdown file will be saved.
    """
    with open(output_path, 'w') as f:
        f.write(markdown_text)

In [5]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [6]:
def prompt_user_for_directory(root, title="Select Directory"):
    # Use a file dialog to get the directory to save markdown files
    markdown_directory = filedialog.askdirectory(
        title=title,
    )
    
    # If user cancels the dialog
    if not markdown_directory:
        print("No directory selected. Exiting.")
        return
    
    return markdown_directory

In [7]:
def calculate_patches(image_file_path):
    """
    Calculate the number of patches in the image file.
    
    Args:
        image_file_path (str): The path to the image file.
    
    Returns:
        int: The number of patches in the image.
    """
    # The number of patches is calculated based on the image size
    # Each patch is 32x32 pixels
    image = Image.open(image_file_path)
    width, height = image.size
    num_x_patches = (width + 32 - 1) // 32 # Ceiling division
    num_y_patches = (height + 32 - 1) // 32 # Ceiling division
    num_patches = num_x_patches * num_y_patches

    return num_patches

In [8]:
# Create a root window
root = tk.Tk()

# Read the OpenAI API key from the text file
openai_key_file_path = os.path.join(os.getcwd(), "openai_key.txt")
if os.path.exists(openai_key_file_path):
    with open(openai_key_file_path, 'r') as key_file:
        openai_api_key = key_file.read().strip()
else:
    print("API key file not found. Please provide the API key manually.")
    openai_api_key = ""

# Prompt the user to provide OpenAI API key using Tkinter
openai_api_key = tk.simpledialog.askstring(
    "OpenAI API Key",
    "Enter your OpenAI API key:",
    initialvalue=openai_api_key
)
if not openai_api_key:
    print("No API key provided. Exiting.")
    root.destroy()
    exit(1)

client = openai.OpenAI(api_key=openai_api_key)

# Destroy the root window
root.destroy()

In [9]:
# Create a root window
root = tk.Tk()

# Using TKinter, prompt the user to provide Google Cloud API json key
google_cloud_key_file_path = os.path.join(os.getcwd(), "google_cloud_key.json")
if os.path.exists(google_cloud_key_file_path):
    with open(google_cloud_key_file_path, 'r') as key_file:
        google_cloud_api_key = key_file.read().strip()
else:
    print("Google Cloud API key file not found. Please provide the API key manually.")
    google_cloud_api_key = ""

# Prompt the user to provide Google Cloud API key using Tkinter file dialog
google_cloud_key_file_path = filedialog.askopenfilename(
    title="Select Google Cloud API Key File",
    filetypes=[("JSON files", "*.json")],
    initialdir=os.getcwd()
)

# Set the environment variable for Google Cloud API key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = google_cloud_key_file_path

# Destroy the root window
root.destroy()

In [10]:
# Create a root window
root = tk.Tk()

# Prompt the user for the file path of the image files
image_directory = prompt_user_for_directory(root, "Select Directory with Image Files")
if not image_directory:
    print("No directory selected. Exiting.")
    root.destroy()
    exit()

markdown_directory = image_directory  # Save markdown files in the same directory as images

# Destroy the root window
root.destroy()

In [11]:
# Get the list of image files in the directory
image_files = [f for f in os.listdir(image_directory) if f.endswith(('.jpg', '.jpeg', '.png'))]
if not image_files:
    print("No image files found in the selected directory.")

# Sort the image files by the number in the filename
image_numbers = {}
for image_file in image_files:
    # Extract the number from the filename
    number = ''.join(filter(str.isdigit, image_file))
    if number:
        image_numbers[image_file] = int(number)
image_files.sort(key=lambda x: image_numbers.get(x, float('inf')))
# Print the sorted image files
print("Sorted image files:")
for image_file in image_files:
    print(image_file)

Sorted image files:
page_139.jpg


In [12]:
def convert_ocr_text_and_vertices_to_markdown(ocr_text, vertices_list, client, log_dir):
    """
    Sends both the OCR text and the vertices list to ChatGPT, instructing it to only arrange the OCR text into markdown,
    not to transcribe from the image.
    """
    # Convert vertices list to string
    vertices_str = "\n".join([str(vertices) for vertices in vertices_list])

    prompt = (
        "You are given the OCR-extracted text from an image and the vertices list on the image of each line of text. "
        "Your task is to arrange the provided OCR text into markdown format, preserving any tables or structure. "
        "Only use the provided OCR text. "
        "Join lines of text that are part of the same row and make sense of the text. "
        "Add headers, line breaks, and other markdown formatting as needed. "
        "If the OCR text is unclear, leave it as is. "
        "Do not add, guess, or hallucinate any information. "
        "Determine if a table is present in the text. "
        "If a table is present, arrange the text into a markdown table, otherwise, return the text as is. "
        "- If a table contains blank or empty cells, preserve them as blank in the markdown table (do not fill or merge them). "
        "- Keep the table structure and number of columns/rows as in the original text, even if some cells are empty. "
        "- The table columns are: ID, Name, Crew, Daily Output, Labor-Hours, Unit, Material, Labor, Equipment, Total, Total Incl O&P. "
        "The vertices list is provided for reference, but do not use it to extract or transcribe any new information. "
        "Use the vertices list only to understand the structure of the text. "
        "Provide only the markdown output, without any extra commentary or code blocks.\n\n"
        "OCR Text:\n"
        f"{ocr_text}\n\n"
        "Vertices List:\n"
        f"{vertices_str}"
    )
    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ]
            }
        ],
        temperature=0,
    )
    markdown_text = response.choices[0].message.content.strip()
    # Logging
    if log_dir is not None:
        log_file_path = os.path.join(log_dir, "conversion_log.txt")
        with open(log_file_path, 'a', encoding='utf-8') as log_file:
            log_file.write(f"Timestamp: {datetime.datetime.now().isoformat()}\n")
            log_file.write("OCR Text:\n")
            log_file.write(ocr_text + "\n")
            log_file.write("Prompt:\n")
            log_file.write(prompt + "\n")
            log_file.write("GPT Markdown Output:\n")
            log_file.write(markdown_text + "\n")
            log_file.write("="*60 + "\n\n")
    return markdown_text


In [13]:
def process_vertices_list(vertices_list):
    """
    Process the vertices list to extract the bounding boxes of each line of text.
    """
    processed_vertices = []
    for line in vertices_list:
        processed_line = []
        for vertex in line:
            x = float(vertex.strip("()").split(",")[0])
            y = float(vertex.strip("()").split(",")[1])
            processed_line.append((x, y))
        processed_vertices.append(processed_line)
    return processed_vertices
            

In [14]:
def plot_vertices_list(image_path, vertices_list):
    """
    Plot the vertices list on the image.
    """
    image = cv2.imread(image_path)
    for line in vertices_list:
        for vertex in line:
            x = int(vertex[0])
            y = int(vertex[1])
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
    cv2.imshow("Vertices", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the image with vertices
    output_path = os.path.splitext(image_path)[0] + "_vertices.jpg"
    cv2.imwrite(output_path, image)
    print(f"Vertices image saved to {output_path}")
    return output_path

In [15]:
def preprocess_text(text, vertices_list):
    """
    Preprocess the text to remove unwanted strings
    and group lines of text.
    """
    processed_lines = []

    num_lines = len(text.split('\n'))
    num_vertices = len(vertices_list)

    print(f"Number of lines: {num_lines}")
    print(f"Number of vertices: {num_vertices}")

    # return processed_text

In [23]:
# Create a root window
root = tk.Tk()
try:
    # Loop through each image file and convert it to markdown
    # for image_file in image_files[:1]: # Limit to 1 file for testing
    for image_file in image_files:
        image_file_path = os.path.join(image_directory, image_file)

        # Check if the markdown file already exists
        markdown_file_name = os.path.splitext(image_file)[0] + ".md"
        markdown_file_path = os.path.join(markdown_directory, markdown_file_name)

        if os.path.exists(markdown_file_path):
            print(f"Markdown file {markdown_file_name} already exists. Skipping conversion.")
            continue

        # Calculate number of patches
        num_patches = calculate_patches(image_file_path)
        print(f"Number of patches in {image_file}: {num_patches}")

        # Confirm with the user before proceeding
        proceed = tk.messagebox.askyesno("Proceed?", f"Do you want to convert {image_file} to markdown?\nNumber of patches: {num_patches}")
        if not proceed:
            print("Conversion cancelled by user.")
            continue
        
        # Check if the file has already been processed with OCR
        ocr_text_file_path = os.path.join(markdown_directory, f"{os.path.splitext(image_file)[0]}_ocr.txt")
        processed_ocr_text_file_path = os.path.join(markdown_directory, f"{os.path.splitext(image_file)[0]}_processed_ocr.txt")

        if os.path.exists(ocr_text_file_path):
            print(f"OCR text file {ocr_text_file_path} already exists. Skipping OCR.")
            with open(ocr_text_file_path, 'r') as ocr_file:
                ocr_text = ocr_file.read()

            # Check if the vertices file exists
            vertices_file_path = os.path.join(markdown_directory, f"{os.path.splitext(image_file)[0]}_vertices.txt")
            if os.path.exists(vertices_file_path):
                print(f"Vertices file {vertices_file_path} already exists. Skipping vertices extraction.")
                with open(vertices_file_path, 'r') as vertices_file:
                    vertices_list = [line.strip() for line in vertices_file.readlines()]
                
                # Each line of the vertices list should be a list of four strings
                vertices_list = [eval(vertices) for vertices in vertices_list]
                print(f"Loaded vertices list from {vertices_file_path}")
                # print(vertices_list[0])

            else:
                # Extract text using OCR
                ocr_text, vertices_list = extract_text_with_google_vision(image_file_path)

                # Save the vertices list to a file
                with open(vertices_file_path, 'w', encoding='utf-8') as vertices_file:
                    for vertices in vertices_list:
                        vertices_file.write(f"{vertices}\n")
                print(f"Saved vertices list to {vertices_file_path}")

        else:
        # if True: # Forcing OCR processing for testing
            # Extract text using OCR
            print(f"Extracting text from {image_file} using OCR...")
            ocr_text, vertices_list = extract_text_with_google_vision(image_file_path)

            # Save the OCR text to a file
            with open(ocr_text_file_path, 'w', encoding='utf-8') as ocr_file:
                ocr_file.write(ocr_text)
            print(f"Saved OCR text to {ocr_text_file_path}")

            # Save the vertices list to a file
            vertices_file_path = os.path.join(markdown_directory, f"{os.path.splitext(image_file)[0]}_vertices.txt")
            with open(vertices_file_path, 'w', encoding='utf-8') as vertices_file:
                for vertices in vertices_list:
                    vertices_file.write(f"{vertices}\n")
            print(f"Saved vertices list to {vertices_file_path}")

            # print(vertices_list[0])
        
        # # Process the vertices list to change the format to four tuples of floats
        # vertices_list = process_vertices_list(vertices_list)
        # print(f"Processed vertices list: {vertices_list[0]}")

        # Plot the vertices list on the image
        # plot_vertices_list(image_file_path, vertices_list)

        print(f"type(ocr_text): {type(ocr_text)}")

        # Preprocess the OCR text using the vertices list
        ocr_text = preprocess_text(ocr_text, vertices_list)
        print(f"Preprocessed OCR text: {ocr_text}")

        # # Send the OCR text and the vertices to ChatGPT for markdown conversion
        # markdown_text = convert_ocr_text_and_vertices_to_markdown(
        #     ocr_text,
        #     vertices_list,
        #     client,
        #     log_dir=markdown_directory
        # )

        # # Save the markdown text to a file
        # with open(markdown_file_path, 'w', encoding='utf-8') as markdown_file:
        #     markdown_file.write(markdown_text)

        # # Print the markdown text
        # print(f"Markdown text for {image_file}:\n{markdown_text}")

        # print(f"OCR extracted text for {image_file}:\n{ocr_text}")

    # Print a message indicating that the process is complete
    print("All images converted to markdown successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

# Close the root window
root.destroy()

Number of patches in page_139.jpg: 3283
Extracting text from page_139.jpg using OCR...
Saved OCR text to C:/Users/scleb/Documents/bimsc25/GitHub/PDF-Markdown-AI/250510_test_images\page_139_ocr.txt
Saved vertices list to C:/Users/scleb/Documents/bimsc25/GitHub/PDF-Markdown-AI/250510_test_images\page_139_vertices.txt
type(ocr_text): <class 'str'>
Number of lines: 1099
Number of vertices: 3690
Preprocessed OCR text: None
All images converted to markdown successfully.
